In [1]:
!ls data/

testData_JSON_chunk10.json     trainingData_JSON_chunk3.json
testData_JSON_chunk8.json      trainingData_JSON_chunk4.json
testData_JSON_chunk9.json      trainingData_JSON.zip
testData_JSON.zip	       trainingData_tabular_chunk1.csv
testData_tabular.csv	       trainingData_tabular_chunk2.csv
testData_tabular.zip	       trainingData_tabular_chunk3.csv
trainingData_JSON_chunk1.json  trainingData_tabular_chunk4.csv
trainingData_JSON_chunk2.json  trainingData_tabular.zip


In [3]:
!head data/trainingData_tabular_chunk1.csv

In [4]:
import pandas as pd

In [5]:
df_train_chunk1 = pd.read_csv('data/trainingData_tabular_chunk1.csv')
df_train_chunk2 = pd.read_csv('data/trainingData_tabular_chunk2.csv')
df_train_chunk3 = pd.read_csv('data/trainingData_tabular_chunk3.csv')
df_train_chunk4 = pd.read_csv('data/trainingData_tabular_chunk4.csv')

In [7]:
df_train = pd.concat([df_train_chunk1, df_train_chunk2, df_train_chunk3, df_train_chunk4])
df_train.reset_index(drop=1, inplace=1)

del df_train_chunk1, df_train_chunk2, df_train_chunk3, df_train_chunk4

In [15]:
df_train.gamestate_id = df_train.gamestate_id.astype('uint32')

In [21]:
max_val = df_train.max()

In [25]:
max_val

gamestate_id                     5000000
decision                               1
turn                                  31
opponent.armor                        21
opponent.attack                        7
opponent.hero_card_id               1235
opponent.hp                           30
opponent.special_skill_used            1
opponent.weapon_durability             4
opponent.crystals_all                 10
opponent.crystals_current             10
opponent.deck_count                   26
opponent.fatigue_damage                7
opponent.hand_count                   10
opponent.played_minions_count          7
player.armor                          20
player.attack                          7
player.hero_card_id                 1235
player.hp                             30
player.special_skill_used              1
player.weapon_durability               4
player.crystals_all                   10
player.crystals_current               10
player.deck_count                     26
player.fatigue_d

In [26]:
for c, v in zip(max_val.index, max_val):
    if c == 'gamestate_id':
        continue
    
    if v <= 255:
        df_train[c] = df_train[c].astype('uint8')
    elif v <= 65535:
        df_train[c] = df_train[c].astype('uint16')
    elif v <= 4294967295:
        df_train[c] = df_train[c].astype('uint32')
    

In [29]:
import feather

In [30]:
feather.write_dataframe(df_train, 'data/df_train.feather')

In [32]:
import xgboost as xgb

In [34]:
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import KFold

In [48]:
xgb_pars = {
    'eta': 0.3,
    'gamma': 0,
    'max_depth': 6,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'lambda': 1,
    'alpha': 0,
    'tree_method': 'approx',
# not deafauts
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 8,
    'seed': 42,
    'silent': 1
}

In [41]:
to_exlude = ['decision', 'gamestate_id']

features = list(df_train.columns)
for c in to_exlude:
    features.remove(c)

In [39]:
y_full = df_train.decision.values
X_full = df_train[columns].values

In [46]:
from tqdm import tqdm
import numpy as np

In [43]:
cv = KFold(len(y_full), n_folds=5, shuffle=True, random_state=42)

In [44]:
n_estimators = 100

In [49]:
#train_preds = np.zeros(y_full.shape)

scores = []

for train, val in tqdm(cv):
    dtrain = xgb.DMatrix(X_full[train], label=y_full[train], feature_names=features, missing=np.nan)
    dval = xgb.DMatrix(X_full[val], label=y_full[val], feature_names=features, missing=np.nan)
    watchlist = [(dtrain, 'train'), (dval, 'val')]

    model = xgb.train(xgb_pars, dtrain, num_boost_round=n_estimators, verbose_eval=10,
                      evals=watchlist)

    y_pred = model.predict(dval)
    #train_preds[val] = y_pred
    score = roc_auc_score(y_full[val], y_pred)

scores.append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

[0]	train-auc:0.757222	val-auc:0.755937
[10]	train-auc:0.787723	val-auc:0.785797
[20]	train-auc:0.795881	val-auc:0.793691
[30]	train-auc:0.800174	val-auc:0.797433
[40]	train-auc:0.803387	val-auc:0.800316
[50]	train-auc:0.80581	val-auc:0.802362
[60]	train-auc:0.807669	val-auc:0.803827
[70]	train-auc:0.809491	val-auc:0.805192
[80]	train-auc:0.810923	val-auc:0.806303
[90]	train-auc:0.812483	val-auc:0.807487


 20%|██        | 1/5 [02:03<08:12, 123.12s/it]

[0]	train-auc:0.756604	val-auc:0.755955
[10]	train-auc:0.788057	val-auc:0.787118
[20]	train-auc:0.795884	val-auc:0.794576
[30]	train-auc:0.800246	val-auc:0.798456
[40]	train-auc:0.803324	val-auc:0.801083
[50]	train-auc:0.805755	val-auc:0.803095
[60]	train-auc:0.80769	val-auc:0.804629
[70]	train-auc:0.809417	val-auc:0.806005
[80]	train-auc:0.810838	val-auc:0.806997
[90]	train-auc:0.81215	val-auc:0.807917


 40%|████      | 2/5 [04:08<06:11, 123.92s/it]

...

 60%|██████    | 3/5 [06:12<04:07, 123.68s/it]

[0]	train-auc:0.758165	val-auc:0.758716
[10]	train-auc:0.788017	val-auc:0.788106
[20]	train-auc:0.795831	val-auc:0.795148
[30]	train-auc:0.799865	val-auc:0.798623
[40]	train-auc:0.803157	val-auc:0.801511
[50]	train-auc:0.805786	val-auc:0.803682
[60]	train-auc:0.807933	val-auc:0.805407
[70]	train-auc:0.809653	val-auc:0.806721
[80]	train-auc:0.811018	val-auc:0.807646
[90]	train-auc:0.81259	val-auc:0.808769


 80%|████████  | 4/5 [08:16<02:03, 123.79s/it]

[0]	train-auc:0.757251	val-auc:0.756025
[10]	train-auc:0.788237	val-auc:0.786583
[20]	train-auc:0.796457	val-auc:0.794033
[30]	train-auc:0.800856	val-auc:0.797906
[40]	train-auc:0.803678	val-auc:0.80027
[50]	train-auc:0.806208	val-auc:0.802244
[60]	train-auc:0.808234	val-auc:0.803853
[70]	train-auc:0.810139	val-auc:0.805407
[80]	train-auc:0.811742	val-auc:0.806662
[90]	train-auc:0.813192	val-auc:0.80776


100%|██████████| 5/5 [10:19<00:00, 123.65s/it]


In [50]:
np.mean(scores), np.std(scores)

(0.80855298014264598, 0.0)

In [51]:
dtrain = xgb.DMatrix(X_full, label=y_full, feature_names=features, missing=np.nan)
watchlist = [(dtrain, 'train')]

model = xgb.train(xgb_pars, dtrain, num_boost_round=n_estimators, verbose_eval=10,
                    evals=watchlist)

[0]	train-auc:0.7571
[10]	train-auc:0.787735
[20]	train-auc:0.795979
[30]	train-auc:0.79994
[40]	train-auc:0.803048
[50]	train-auc:0.805347
[60]	train-auc:0.807241
[70]	train-auc:0.809023
[80]	train-auc:0.810535
[90]	train-auc:0.811647


In [52]:
!ls data/

df_train.feather	       trainingData_JSON_chunk3.json
testData_JSON_chunk10.json     trainingData_JSON_chunk4.json
testData_JSON_chunk8.json      trainingData_JSON.zip
testData_JSON_chunk9.json      trainingData_tabular_chunk1.csv
testData_JSON.zip	       trainingData_tabular_chunk2.csv
testData_tabular.csv	       trainingData_tabular_chunk3.csv
testData_tabular.zip	       trainingData_tabular_chunk4.csv
trainingData_JSON_chunk1.json  trainingData_tabular.zip
trainingData_JSON_chunk2.json


In [53]:
df_test = pd.read_csv('data/testData_tabular.csv')

In [57]:
X_test = df_test[features].astype('uint16').values

In [58]:
dtest = xgb.DMatrix(X_test, feature_names=features, missing=np.nan)

In [60]:
pred = model.predict(dtest)

In [61]:
result = pd.DataFrame()
result['gamestate_id'] = df_test.gamestate_id.astype('uint32')
result['pred'] = pred

In [63]:
result.to_csv('results.csv', index=False)

In [64]:
!head results.csv

gamestate_id,pred
6845857,0.4105454981327057
9279433,0.9685745239257812
6642076,0.7363542318344116
9169516,0.343049019575119
8991397,0.24146009981632233
7681425,0.10094859451055527
9105278,0.5462976694107056
5374115,0.49427488446235657
6685724,0.5552678108215332


In [65]:
!gzip results.csv

In [70]:
with open('results.txt', 'w') as f:
    for p in pred:
        f.write('%.5f\n' % p)

In [71]:
!gzip results.txt